# Profiling ANDES in Notebook

## Profiling with Python CProfiler

In [1]:
import andes
from andes.utils.paths import get_case

case_path = get_case('kundur/kundur_full.xlsx')

Passing `profile=True, no_output = True` to `run` will enable the profiler and have the results printed.

In [2]:
ss = andes.run(case_path, profile=True, routine='tds', no_output=True)

Working directory: "/Users/hcui7/repos/andes/examples"
Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"
Input file parsed in 0.1158 seconds.

-> Power flow calculation
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60858   
2: |F(x)| = 0.17009   
3: |F(x)| = 0.00203822
4: |F(x)| = 3.76399e-07
Converged in 5 iterations in 0.0111 seconds.

-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0.0-20.0sec.
Fixed step size: h=33.33msec., shrink if not converged
Initialization was successful in 0.0159 seconds.


<Toggle 1>: Status of Line.Line_8 changed to 0.0 at t=2.0sec.         
100%|████████████████████████████████| 100/100 [00:01<00:00, 63.85%/s]

Simulation completed in 1.5667 seconds.

-> Single process finished in 1.8691 seconds.



         1272709 function calls (1267945 primitive calls) in 1.850 seconds

   Ordered by: cumulative time
   List reduced from 2079 to 40 due to restriction <40>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.005    0.005    1.630    1.630 /Users/hcui7/repos/andes/andes/routines/tds.py:144(run)
      600    0.083    0.000    1.535    0.003 /Users/hcui7/repos/andes/andes/routines/tds.py:256(_itm_step)
     1834    0.007    0.000    1.233    0.001 /Users/hcui7/repos/andes/andes/routines/tds.py:244(_fg_update)
     9227    0.075    0.000    1.175    0.000 /Users/hcui7/repos/andes/andes/system.py:872(call_models)
     1839    0.002    0.000    0.641    0.000 /Users/hcui7/repos/andes/andes/system.py:582(g_update)
    18370    0.208    0.000    0.613    0.000 /Users/hcui7/repos/andes/andes/core/model.py:1083(g_update)
     1839    0.002    0.000    0.285    0.000 /Users/hcui7/repos/andes/andes/system.py:568(f_update)
    18370    0.120    0.000    0

## Profiling with `line_profiler`.

`line_profiler` provides line-based profiling results for functions. 

Install with `pip install line_profiler` and restart the notebook.

In [3]:
import andes
from andes.utils.paths import get_case

case_path = get_case('kundur/kundur_full.xlsx')

### Profile power flow 

Pass the function name to profile to the magic `%lprun`, followed by a call to the function itself or an upper-level function.

Results will be shown in a popup window.

In [4]:
%lprun -f andes.routines.pflow.PFlow.run andes.run(case_path, no_output=True)

Working directory: "/Users/hcui7/repos/andes/examples"
Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"
Input file parsed in 0.1106 seconds.

-> Power flow calculation
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60858   
2: |F(x)| = 0.17009   
3: |F(x)| = 0.00203822
4: |F(x)| = 3.76399e-07
Converged in 5 iterations in 0.0099 seconds.
-> Single process finished in 0.1719 seconds.


Alternatively, do

In [5]:
ss = andes.run(case_path, no_output=True)

Working directory: "/Users/hcui7/repos/andes/examples"
Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"
Input file parsed in 0.0565 seconds.

-> Power flow calculation
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60858   
2: |F(x)| = 0.17009   
3: |F(x)| = 0.00203822
4: |F(x)| = 3.76399e-07
Converged in 5 iterations in 0.0092 seconds.
-> Single process finished in 0.0995 seconds.


In [6]:
%lprun -f ss.PFlow.run ss.PFlow.run()


-> Power flow calculation
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60858   
2: |F(x)| = 0.17009   
3: |F(x)| = 0.00203822
4: |F(x)| = 3.76399e-07
Converged in 5 iterations in 0.0116 seconds.


To dig into the Newton Raphson iteration steps, profile each step instead with:

In [7]:
%lprun -f ss.PFlow.nr_step ss.PFlow.run()


-> Power flow calculation
Method: NR method
Power flow initialized.
0: |F(x)| = 14.9283   
1: |F(x)| = 3.60858   
2: |F(x)| = 0.17009   
3: |F(x)| = 0.00203822
4: |F(x)| = 3.76399e-07
Converged in 5 iterations in 0.0104 seconds.


### Profile time-domain simulation

In [8]:
%lprun -f ss.TDS._itm_step ss.TDS.run()


-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0.0-20.0sec.
Fixed step size: h=33.33msec., shrink if not converged
Initialization was successful in 0.0191 seconds.


<Toggle 1>: Status of Line.Line_8 changed to 0.0 at t=2.0sec.         
100%|████████████████████████████████| 100/100 [00:01<00:00, 52.32%/s]

Simulation completed in 1.9118 seconds.


## Cleanup

In [9]:
!andes misc -C


    _           _         | Version 0.8.8.post42+gcfb26c4a
   /_\  _ _  __| |___ ___ | Python 3.7.1 on Darwin, 05/02/2020 01:04:54 PM
  / _ \| ' \/ _` / -_|_-< | 
 /_/ \_\_||_\__,_\___/__/ | This program comes with ABSOLUTELY NO WARRANTY.

No output file found in the working directory.
